# Twisted Molybdenum Disulfide bilayers at various angles.

## 0. Introduction.

This notebook demonstrates how to generate a twisted interface between two materials using commensurate lattices. The example uses molybdenum disulfide (MoS2) as the film and substrate. The twisted interface is generated by rotating the film with respect to the substrate by a target twist angle. The algorithm searches for a supercell matrix that aligns the film and substrate lattices within a specified angle tolerance. The generated interface is visualized and analyzed to determine the actual twist angle and the number of atoms in the interface.

> **Kaihui Liu, Liming Zhang, Ting Cao, Chenhao Jin, Diana Qiu, Qin Zhou, Alex Zettl, Peidong Yang, Steve G. Louie & Feng Wang**
> Evolution of interlayer coupling in twisted molybdenum disulfide bilayers. Nature Communications, 5, 4966. 2014.
 > [https://doi.org/10.1038/ncomms5966](https://doi.org/10.1038/ncomms5966)

The twisted MoS2 bilayers are shown in the following Figure 4 from the article.

<img src="https://i.imgur.com/73jHlWu.png" alt="Twisted MoS2 bilayers" width="600"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook
Let's set angles and corresponding distances for the twisted interface from the article.

In [ ]:
# Uncomment lines to reproduce specific cases from the article
INTERFACE_PARAMETERS = [
    # {"angle": 0.0, "distance": 6.8},
    # {"angle": 13.0, "distance": 6.5},
    {"angle": 22.0, "distance": 6.5},
    # {"angle": 38.0, "distance": 6.5},
    # {"angle": 47.0, "distance": 6.5},
    # {"angle": 60.0, "distance": 6.2},
]

INTERFACE_VACUUM = 20.0  # in Angstroms

# Search algorithm parameters
MAX_REPETITION = None  # Maximum supercell matrix element value (None for automatic)
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

 ### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
We'll use the MoS2 material from Standata.


In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from utils.visualize import visualize_materials

material = Material(Materials.get_by_name_and_categories("MoS2", "2D"))

print("Initial material properties:")
print(f"Formula: {material.formula}")
print(f"Number of atoms: {len(material.basis.elements.ids)}")

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS)
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS, rotation="-90x")


 ## 3. Generate Twisted Interface
 ### 3.1. Set up interface configuration and builder


In [ ]:
from mat3ra.made.tools.build.interface.builders import CommensurateLatticeTwistedInterfaceBuilderParameters, \
    CommensurateLatticeTwistedInterfaceBuilder
from mat3ra.made.tools.build.interface.configuration import TwistedInterfaceConfiguration

configs = [TwistedInterfaceConfiguration(
    film=material,
    substrate=material,
    twist_angle=parameters["angle"],
    distance_z=parameters["distance"],
    vacuum=INTERFACE_VACUUM
) for parameters in INTERFACE_PARAMETERS]

params = CommensurateLatticeTwistedInterfaceBuilderParameters(
    max_repetition_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH
)

builder = CommensurateLatticeTwistedInterfaceBuilder(build_parameters=params)

### 3.2. Generate and analyze interfaces


In [ ]:
interfaces = [builder.get_material(config) for config in configs]

## 4. Preview the  materials


In [ ]:
from utils.visualize import visualize_materials

for interface in interfaces:
    visualize_materials(interface, repetitions=VISUALIZE_REPETITIONS)


### 5. Download materials


In [ ]:
from utils.jupyterlite import download_content_to_file

download_content_to_file(interfaces, "twisted_interfaces.json")